In [1]:
from stage import getStage, fullPrint
import pandas as pd
import datetime
from datetime import date, datetime
import yahoo_fin.stock_info as yf

In [2]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
goodSectorDf.index = sectorDfList[0].index
index = 0
for date in sectorDfList[0].index:
    listOfSector = []
    for df in sectorDfList:
        try:
            #print(df.at[date,'MA30Slope'],df.at[date,'RS'])
            #!!!!!!!! or for new params
            #!!!!!!!!
            #!!!!!!!!
            #!!!!!!!!
            if  df.at[date,'MA30Slope'] > 0.992549883242513 and df.loc[date,'RS'] > -0.3044455098712273:
                if df.at[date,'close'] > df.at[date,'MA30']:
                    #print(df.at[date,'ticker'])
                    listOfSector.append(df.at[date,'ticker'])
        except: 
            pass
        # print(listOfSector)
    goodSectorDf.iat[index,0] = set(listOfSector)
    index = index + 1
    #print(goodSectorDf.at[date,'Sectors'])
fullPrint(goodSectorDf)

                                                      Sectors
1999-01-08                                                 {}
1999-01-15                                                 {}
1999-01-22                                                 {}
1999-01-29                                                 {}
1999-02-05                                                 {}
1999-02-12                                                 {}
1999-02-19                                                 {}
1999-02-26                                                 {}
1999-03-05                                                 {}
1999-03-12                                                 {}
1999-03-19                                                 {}
1999-03-26                                                 {}
1999-04-09                                                 {}
1999-04-16                                                 {}
1999-04-23                                                 {}
1999-04-

In [3]:
removeList = []
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
for stock in nasdaqList:
    try:
        df = getStage(stock,[0.10684256968972883, 1.1421709856192384, 0.27350824408318475, 1.0231080328602544, 1.0760547872425865, 1.2745231615209662, 0.9038554056254413, 1.0764846753295985, 1.2709398376771783, 0.992549883242513, -0.3044455098712273, 0.9467020070162686, 1.0014235121991226, 0.8164237632733558, 0.9867262709280287],goodSectorDf)
        if df.empty:
            removeList.append(stock)
    except:
        removeList.append(stock)

In [4]:
import pickle
for removeItem in removeList:
    nasdaqList.remove(removeItem)
listOfTickerObject = open('stockData/tickerListpkl', 'wb')
pickle.dump(nasdaqList, listOfTickerObject)
listOfTickerObject.close()

In [5]:
# Open a file
result = open("Testdata/Realtime/result"+date.today().strftime('%Y-%m-%d')+".txt", "w")
buyList = []
holdList = []
sellList = []
for symbol in nasdaqList:
    df = getStage(symbol,[0.10684256968972883, 1.1421709856192384, 0.27350824408318475, 1.0231080328602544, 1.0760547872425865, 1.2745231615209662, 0.9038554056254413, 1.0764846753295985, 1.2709398376771783, 0.992549883242513, -0.3044455098712273, 0.9467020070162686, 1.0014235121991226, 0.8164237632733558, 0.9867262709280287],goodSectorDf)
    shares = 0
    sharesSPY = 0
    if df.empty:
        continue
    if df.iloc[-1].Stage=="Buy":
        print(symbol+" buy " + df.iloc[-1].support)
        result.write(symbol+" buy " + df.iloc[-1].support + "\n")
        buyList.append(symbol)
    elif df.iloc[-1].Stage=="Stage 2":
        print(symbol+" hold " + df.iloc[-1].support)
        result.write(symbol+" hold " + df.iloc[-1].support + "\n")
        holdList.append(holdList)
    elif df.iloc[-1].Stage[0:4]=="Sell":
        print(symbol+" sell")
        result.write(symbol+" sell\n")
        sellList.append(sellList)
result.close()

AACG
AACI
AACIU
AADI
AAL
AAME
AAOI
AAON
AAPL
AATC
AAWW
ABCB
ABCL
ABCM
ABEO
ABGI
ABIO
ABMD
ABNB
ABOS
ABSI
ABST
ABTX
ABUS
ABVC
ACABU
ACAD
ACAH
ACAHU
ACAXU
ACB
ACBA
ACBAU
ACCD
ACER
ACET
ACEV
ACEVU
ACGL
ACGLN
ACGLO
ACHC
ACHL
ACHV
ACIU
ACIW
ACLS
ACLX
ACMR
ACNB
ACOR
ACQR
ACQRU
ACRS
ACRX
ACST
ACT
ACTD
ACTDU
ACTG
ACVA
ACXP
ADAG
ADAL
ADALU
ADALW
ADAP
ADBE
ADER
ADERU
ADES
ADGI
ADI
ADIL
ADMA
ADMP
ADN
ADOC
ADP
ADPT
ADSE
ADSEW
ADSK
ADTH
ADTHW
ADTN
ADTX
ADUS
ADV
ADVM
ADXN
AEAC
AEACU
AEAE
AEAEU
AEAEW
AEHA
AEHAU
AEHL
AEHR
AEI
AEIS
AEMD
AEP
AEPPZ
AERC
AERI
AESE
AEY
AEYE
AEZS
AFAC
AFACU
AFAQ
AFAQU
AFBI
AFCG
AFIB
AFMD
AFRM
AFYA
AGBA
AGBAU
AGEN
AGFS
AGFY
AGGR
AGGRU
AGIL
AGIO
AGLE
AGMH
AGNC
AGNCM
AGNCN
AGNCO
AGNCP
AGRI
AGRI buy
AGRIW
AGRX
AGTC
AGYS
AHCO
AHG
AHI
AHPA
AHPAU
AHPI
AIH
AIHS
AIKI
AIMC
AINV
AIP
AIRG
AIRS
AIRT
AIRTP
AKAM
AKBA
AKIC
AKICU
AKRO
AKTS
AKTX
AKU
AKUS
AKYA
ALAC
ALACU
ALACW
ALBO
ALCO
ALDX
ALEC
ALF
ALGM
ALGN
ALGS
ALGT
ALHC
ALIM
ALJJ
ALKS
ALKT
ALLK
ALLO
ALLR
ALLT
ALNA
ALNY
ALOR
ALORU
ALORW
A

In [6]:
for symbol in ["AACG"]:
    df = getStage(symbol,[0.10684256968972883, 1.1421709856192384, 0.27350824408318475, 1.0231080328602544, 1.0760547872425865, 1.2745231615209662, 0.9038554056254413, 1.0764846753295985, 1.2709398376771783, 0.992549883242513, -0.3044455098712273, 0.9467020070162686, 1.0014235121991226, 0.8164237632733558, 0.9867262709280287],goodSectorDf)
    df.to_csv("realtime"+symbol+".csv")

In [7]:
print(buyList)

['AGRI', 'ARTW', 'CPLP', 'HBP', 'PNBK', 'RADA', 'SAMG', 'SND']
